# Vanilla LSTM for Gene/No gene classification
The Milestone 1 corresponds to the classication task of, given a sequence, predict if it contains a gene, a partial sequence of a gene or just intergenic code.

In [1]:
!pip3 install pyfastx

In [0]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import pandas as pd

from tqdm import tqdm # progress bar
from preproc_pipeline import window_pipeline
from warnings import simplefilter

In [187]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Dataset for training
The genome of E. coli will be used for this purpose.

In [0]:
simplefilter("ignore")
genome = "GCF_000008865.2_ASM886v2_genomic.fna"
feature_table = "GCA_000008865.2_ASM886v2_feature_table.tsv"
df = window_pipeline(genome, feature_table)

In [5]:
print(df.sequence.apply(lambda x: len(x)).max())
len(df)

50


36526

In [6]:
print(
    f"columns -> {list(df.columns)}\n"
    f"labels in dataframe -> {list(df.label.unique())}"
)

columns -> ['sequence', 'label']
labels in dataframe -> ['gene', 'intergenic', 'partial']


Let's get a one hot mapping for the labels.

In [7]:
labels = list(df.label.unique())
lab0 = np.zeros(len(labels))
lab2vec = {}
vec2lab = {}
for i, label in enumerate(list(df.label.unique())):
    labv = lab0.copy()
    labv[i] = 1
    lab2vec[label] = labv
    vec2lab[tuple(labv)] = label

print(f"lab2vec -> {lab2vec}\nvec2lab -> {vec2lab}")

lab2vec -> {'gene': array([1., 0., 0.]), 'intergenic': array([0., 1., 0.]), 'partial': array([0., 0., 1.])}
vec2lab -> {(1.0, 0.0, 0.0): 'gene', (0.0, 1.0, 0.0): 'intergenic', (0.0, 0.0, 1.0): 'partial'}


In [8]:
print(df[df.sequence.apply(lambda x: len(x)==0)].count())
df = df[~df.sequence.apply(lambda x: len(x)==0)]

sequence    0
label       0
dtype: int64


Need to check why it always generate a 0 length row. I think is the last one, but I am not sure.

In [15]:
df["label_onehot"] = df.label.apply(lambda x: lab2vec[x])
toy = pd.concat([df[df.label=="gene"].sample(n=100),
                df[df.label=="intergenic"].sample(n=100),
                df[df.label=="partial"].sample(n=100)]).reset_index(drop=True)
df_train = toy.sample(frac=8/10) # shuffle
df_test = df[~df.index.isin(df_train.index)].dropna().reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_train

,sequence,label,label_onehot
0,TGATTCGCTTAGCGCCCTTGATTACCGCCGACGTTGACACCACCAC...,gene,"[1.0, 0.0, 0.0]"
1,CTGAATAATGAGTAAAAGGAGAGAGAGCCGAACGGGAGATAATTCG...,intergenic,"[0.0, 1.0, 0.0]"
2,ATCAAAGTGAATTTTGCGTCACGATCGGTGCATCAAGCCGAGGAGT...,intergenic,"[0.0, 1.0, 0.0]"
3,TGCAGCCCAAAATTTACTGGATTGATAACCTGCGAGGGATAGCGTG...,gene,"[1.0, 0.0, 0.0]"
4,CAGTGCTCAGTCATGAACACTCTGTTTCAGTGAGCAGTTTCCGCAT...,partial,"[0.0, 0.0, 1.0]"
...,...,...,...
235,ACAAGGTGACAATCAATAACAAAATGCGGCGTAAGGTGAAGATAAT...,gene,"[1.0, 0.0, 0.0]"
236,CTGGCGGCGATCAATGAAATTAGCGAAAGCGAAGTTCATCGCAGCC...,partial,"[0.0, 0.0, 1.0]"
237,CAGGAAAAGGTACCTGGCGAATGTTGCGCAAACTGATGTGGCGTTA...,gene,"[1.0, 0.0, 0.0]"
238,AAGCGGAGCCGCTCTTTGTAACAACGGACATTCGTCCTACGTCGCT...,intergenic,"[0.0, 1.0, 0.0]"


## 2. Embeddings
The next step is to use the whole sequence to compute the embeddings. First, get a set of k-mers, that will be our words for this NLP problem.

In [0]:
def window(fseq, window_size, slide = 1):
    # create a window of size k
    N = len(fseq)
    for i in range(0, N - window_size + 1, slide):
      if i+window_size+slide < N:
        yield fseq[i:i+window_size]


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]

class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim, padding_idx):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, 
                                       padding_idx=padding_idx) #used predefined nn.Embedding
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.LongTensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)
    
k = 3                     # so our words has length 3
SLIDE = 1                 # sampling slide
CONTEXT_SIZE = 2          # 2 words to the left, 2 to the right
EMDEDDING_DIM = 25        # embedding dimension
EPOCHS = 10               # number of epochs for training
model = None              # CBOW model

Finally, gather all the kmers and apply the CBOW algorithm.

In [0]:
# Since we have the embeddings stored, we are going to ignore the following next
# two cells and use this one
stored = False

if stored:
  with open("wti.p", "rb") as f:
    word_to_ix = pickle.load(f)
  ix_to_word = {v: k for k,v in word_to_ix.items()}
  model_save_name = 'ma_model.pt'
  path = model_save_name
  #path = F"/content/drive/My Drive/{model_save_name}" 
  model = CBOW(len(word_to_ix), EMDEDDING_DIM, padding_idx=word_to_ix["X"]).cpu()
  model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

In [18]:
# kmer in all of the 1/4 sequences of the dataset
kmers = [ kmer for kmer in window("".join([seq for seq in df_train.loc[:,"sequence"]]), k, SLIDE) ]  
vocab_size = len(kmers)

data = []
print("Filling context data...")
pbar = tqdm(total=vocab_size - 4-1) # just to output something in screen
for i in range(2, vocab_size - 2): # first word to have 2 words before is the "third" one (0,1,2)
    context = (kmers[i - 2], kmers[i - 1],
               kmers[i + 1], kmers[i + 2])
    target = kmers[i]
    data.append((context, target))
    pbar.update(1)

word_to_ix, ix_to_word = {},{}
for i, word in enumerate(set(kmers)):
    word_to_ix[word] = i
    ix_to_word[i] = word

ix = len(ix_to_word)
ix_to_word[ix] = "X"  # our padding character
word_to_ix["X"] = ix

with open("/content/drive/My Drive/wti.p", "wb") as f:
  pickle.dump(word_to_ix, f)

with open("/content/drive/My Drive/itw.p", "wb") as f:
  pickle.dump(ix_to_word, f)


  0%|          | 0/11991 [00:00<?, ?it/s]

Filling context data...


In [19]:
import sys
from time import time 
model = CBOW(len(word_to_ix), EMDEDDING_DIM, padding_idx=word_to_ix["X"])

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
print("Training embeddings...")

model_save_name = 'ma_model.pt'
path = F"/content/drive/My Drive/{model_save_name}" 

start = time()
tot = len(data)
print()
for epoch in range(EPOCHS):
  start = time()
  total_loss = 0
  i = 1
  for context, target in data:
    if not i%100:
      sys.stdout.write(f"\r{i}/{tot} in {round(time()-start,2)}s        ")
      sys.stdout.flush()
    context_vector = make_context_vector(context, word_to_ix)  
    model.zero_grad()
    log_probs = model(context_vector)
    loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
    loss.backward()
    optimizer.step()
    total_loss += loss.data
    i+=1
  torch.save(model.state_dict(), path)
  print(f"\nEpoch {epoch+1}/{EPOCHS} in {time()-start}\n\n")

Training embeddings...

11900/11992 in 9.2s        
Epoch 1/10 in 9.63705039024353


11900/11992 in 9.18s        
Epoch 2/10 in 9.253341913223267


11900/11992 in 9.35s        
Epoch 3/10 in 9.434730768203735


11900/11992 in 10.11s        
Epoch 4/10 in 10.195592880249023


11900/11992 in 9.62s        
Epoch 5/10 in 9.700698852539062


11900/11992 in 9.39s        
Epoch 6/10 in 9.466909170150757


11900/11992 in 9.18s        
Epoch 7/10 in 9.255321502685547


11900/11992 in 9.21s        
Epoch 8/10 in 9.288195610046387


11900/11992 in 9.28s        
Epoch 9/10 in 9.357722282409668


11900/11992 in 9.24s        
Epoch 10/10 in 9.322306394577026




## 4. Vanilla RNN model

In [0]:
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, hidden_out, output_dim,
                 padding_idx, t):

        super().__init__()
        self.nb_tags = output_dim

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=padding_idx)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        
        self.max_seq_length = max_seq_length
        self.lhid = nn.Linear(1564, hidden_out)
        # self.lhid = nn.Linear(hidden_dim*t, hidden_out)

        self.fc = nn.Linear(hidden_out, output_dim)


    def forward(self, text):

        #text = [sent len, batch size]
        # 1. embedding
        embedded = self.embedding(text)

        #embedded = [sent len, batch size, emb dim]
        output, (hidden, lt) = self.rnn(embedded)

        # 3. get that so it's correctly packed for the hidden layer
        output = output.contiguous()
        output = output.reshape(output.shape[0], -1)

        output = nn.functional.relu(self.lhid(output))
        #print('after linear layer')
        #print(output.size())

        # 4. classification
        output = self.fc(output)
        #print(output.size())
        output = nn.functional.log_softmax(output, dim=1)
        #print(output.size())
        # output = output.view(text.size()[0], -1)
        return output

## 4. Tweak the embeddings to accomodate varying sizes of the sequences
Once we have the model and the embeddings, we would need to tweak the embeddings so that they are adjusted for the padded sequences.

### 4.1. Add the padding char to the embeddings

Now, get sequences as indexes.

In [0]:
def to_ix(kmer, word_to_ix):
  if "X" in kmer:
    return word_to_ix["X"]
  elif kmer in word_to_ix:
    return word_to_ix[kmer]
  else:
    return len(word_to_ix)

def wind_idx(seq):
  return [to_ix(kmer, word_to_ix)  for kmer in window(seq, k, SLIDE)]

df_train["seq_idx"] = df_train["sequence"].apply(wind_idx)
df_test["seq_idx"] = df_test["sequence"].apply(wind_idx)

In [43]:
np.unique(df_train["seq_idx"].apply(lambda x: len(x)))

46

Finally, instantiante the model and initialize the weigths of the embeddings.

In [40]:
print(word_to_ix["X"])

64


In [0]:
t = len(df_train.iloc[0,0]) # Find length of seqs (fixed)

rnn = RNN(input_dim=len(word_to_ix), embedding_dim=EMDEDDING_DIM,
          hidden_dim = 34, hidden_out = 90, output_dim=3, 
          padding_idx = word_to_ix["X"], t = len(df_train.iloc[0,0]))

In [63]:
rnn.embedding.weight.data.copy_(model.embeddings.weight)

tensor([[-2.0958, -0.0827,  0.0517,  ..., -0.6931, -1.0605, -0.0412],
        [ 0.6832, -0.4075,  1.6770,  ...,  0.8553, -1.6916, -0.3104],
        [-0.0936, -0.8667,  2.1091,  ...,  0.0122,  1.6149, -0.0303],
        ...,
        [ 0.3009, -0.5067, -1.0054,  ..., -0.0231,  0.8620,  0.1041],
        [-0.8633, -0.5608, -0.2426,  ..., -0.2470,  0.7122, -0.0293],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<CopyBackwards>)

## 5. Training loop


All that is left is to split our training and testing and train the model.

In [0]:
from torch.utils.data import DataLoader, Dataset

class oversampdata(Dataset):
  def __init__(self, data):
    # first column is list of index sentence
    self.data = torch.LongTensor(data.iloc[:,0])
    # second column is the label
    self.targets = torch.LongTensor(data.iloc[:,1])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    data_val = self.data[index]
    target = self.targets[index]
    return data_val, target

train_dataset = oversampdata(df_train.loc[:,["seq_idx", "label_onehot"]])
valid_dataset = oversampdata(df_test.loc[:,["seq_idx", "label_onehot"]])

In [0]:
BATCH_SIZE = 32

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, 
                                         shuffle=False)

In [0]:
def binary_accuracy(preds, y):
   """
   Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
   """
   #round predictions to the closest integer
   rounded_preds = torch.round(torch.sigmoid(preds))
   correct = (rounded_preds == y).float() #convert into float for division
   acc = correct.sum() / len(correct)
   return acc

In [0]:
import sys

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    print("Training...")

    for i, batch in enumerate(iterator):
        if i%50:
          sys.stdout.write(f"\rIteration {i}        ")
          sys.stdout.flush()

        inputs, labels_onehot = batch

        optimizer.zero_grad()
                
        predictions = model(inputs)
        
        labels_idx = torch.LongTensor([np.where(label==1)[0][0] for label in labels_onehot])


        loss = criterion(predictions, labels_idx)
        
        acc = binary_accuracy(predictions, labels_onehot)
        
        loss.backward()
        
        optimizer.step()
        
        print(loss.item())
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    print()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    print("Evaluating...")
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            inputs, labels_onehot = batch
            predictions = model(inputs)
            labels_idx = torch.LongTensor([np.where(label==1)[0][0] for label in labels_onehot])

            loss = criterion(predictions, labels_idx)
            acc = binary_accuracy(predictions, labels_onehot)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

In [55]:
len(train_dataset)

240

In [68]:
%%time
N_EPOCHS = 5
model_save_name="ma_rnn.pt"
path = F"/content/drive/My Drive/{model_save_name}"

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    print("Epoch: " + str(epoch))
    train_loss, train_acc = train(rnn, trainloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(rnn, testloader, criterion)

    #train_loss  = train(rnn, trainloader, optimizer, criterion)
    #valid_loss  = evaluate(rnn, testloader, criterion)    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(rnn.state_dict(), path)
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    
    #print(f'\tTrain Loss: {train_loss:.3f}')
    #print(f'\t Val. Loss: {valid_loss:.3f}')

Epoch: 0
Training...
1.1022065877914429
Iteration 1        1.0999407768249512
Iteration 2        1.1181011199951172
Iteration 3        1.1063838005065918
Iteration 4        1.0966620445251465
Iteration 5        1.1065168380737305
Iteration 6        1.077891230583191
Iteration 7        1.0856735706329346

Evaluating...
	Train Loss: 1.099 | Train Acc: 200.00%
	 Val. Loss: 1.102 |  Val. Acc: 200.00%
Epoch: 1
Training...
1.1048164367675781
Iteration 1        1.1120909452438354
Iteration 2        1.1098743677139282
Iteration 3        1.0967321395874023
Iteration 4        1.0782041549682617
Iteration 5        1.0898891687393188
Iteration 6        1.1022045612335205
Iteration 7        1.0886417627334595

Evaluating...
	Train Loss: 1.098 | Train Acc: 200.00%
	 Val. Loss: 1.102 |  Val. Acc: 200.00%
Epoch: 2
Training...
1.087902545928955
Iteration 1        1.1117578744888306
Iteration 2        1.09014093875885
Iteration 3        1.0935827493667603
Iteration 4        1.1173139810562134
Iteration 